In [1]:
#required imports
import json
import pprint as pp
from datetime import datetime

In [2]:
#loading data
with open("20230706153517_JOURNEY_5N1BT3CB0PC795389_20230706154047_idling1.jsonl",'r') as file:
    data = json.load(file)

In [3]:
#loading data
with open('20230706153517_JOURNEY_5N1BT3CB0PC795389_20230706154047_idling2.jsonl',"r") as file:
    data1 = json.load(file)

In [4]:
ignitionOff = data['ignitionOff']

In [5]:
pp.pprint(ignitionOff,sort_dicts=0)

{'timeStamp': '2023-07-06T15:40:47Z',
 'sourceFile': 'decrypted/vNext_PROBE_06-07-2023_GammaExport_bVhymA_20230706164806.jsonl.gz',
 'LocationLatitude': '32.9161619444444',
 'LocationSourceAtEnd': '1',
 'ECOAverageDistanceForFuelEconomy': '23.8',
 'StatusWheelPressureRearAxle': '210',
 'StatusWheelStateFrontLeft': '0',
 'StatusWheelPressureRearLeft': '2370',
 'StatusWheelStateRearRight': '0',
 'RapidDecelerationAccelerationSpeedatStart1': '0',
 'RapidDecelerationAccelerationSpeedatStart2': '0',
 'RapidDecelerationAccelerationSpeedatStart5': '0',
 'RapidDecelerationAccelerationSpeedatStart3': '0',
 'RapidDecelerationAccelerationSpeedatStart4': '0',
 'AirTempDrivingStartupUnitICE': '0',
 'StatusBrakeLowFluidLevel': '1',
 'RapidDecelerationAccelerationMaximizeSpeedDeceleration5': '0',
 'RapidDecelerationAccelerationMaximizeSpeedDeceleration4': '0',
 'RapidDecelerationAccelerationMaximizeSpeedDeceleration3': '0',
 'RapidDecelerationAccelerationMaximizeSpeedDeceleration2': '0',
 'RapidDecel

In [6]:
#extracting only idling data parts from the ingnitonOff dictionary
idlingData = {}
for keys,values in ignitionOff.items():
    if ('ExcessiveIdlingTimeStamp' in keys) or ('ExcessiveIdlingElapsedTime' in keys):
        idlingData.update({keys:values})

In [7]:
idlingData

{'ExcessiveIdlingElapsedTime1': '120',
 'ExcessiveIdlingTimeStamp1': '7/14/2023 4:40:47 PM',
 'ExcessiveIdlingElapsedTime2': '160',
 'ExcessiveIdlingTimeStamp2': '7/14/2023 4:45:47 PM',
 'ExcessiveIdlingElapsedTime3': '180',
 'ExcessiveIdlingTimeStamp3': '7/14/2023 4:55:47 PM',
 'ExcessiveIdlingElapsedTime4': '120',
 'ExcessiveIdlingTimeStamp4': '7/14/2023 5:15:47 PM',
 'ExcessiveIdlingElapsedTime5': '120',
 'ExcessiveIdlingTimeStamp5': '7/14/2023 5:20:47 PM',
 'ExcessiveIdlingElapsedTime6': '0',
 'ExcessiveIdlingTimeStamp6': '1/1/1973 12:00:00 AM'}

In [8]:
#cleaned the data by removing zero value fields and epoch start time fields
cleaned_idlingData = {}
for keys,values in idlingData.items():
    if ('ExcessiveIdlingElapsedTime' in keys) and (int(values)==0):
        print(f"{keys}\t\t{values}")
    elif ('ExcessiveIdlingTimeStamp' in keys) and ('1/1/1973 12:00:00 AM' in values) :
        print(f"{keys}\t\t{values}")
    else:
        cleaned_idlingData.update({keys:values})

ExcessiveIdlingElapsedTime6		0
ExcessiveIdlingTimeStamp6		1/1/1973 12:00:00 AM


In [9]:
cleaned_idlingData

{'ExcessiveIdlingElapsedTime1': '120',
 'ExcessiveIdlingTimeStamp1': '7/14/2023 4:40:47 PM',
 'ExcessiveIdlingElapsedTime2': '160',
 'ExcessiveIdlingTimeStamp2': '7/14/2023 4:45:47 PM',
 'ExcessiveIdlingElapsedTime3': '180',
 'ExcessiveIdlingTimeStamp3': '7/14/2023 4:55:47 PM',
 'ExcessiveIdlingElapsedTime4': '120',
 'ExcessiveIdlingTimeStamp4': '7/14/2023 5:15:47 PM',
 'ExcessiveIdlingElapsedTime5': '120',
 'ExcessiveIdlingTimeStamp5': '7/14/2023 5:20:47 PM'}

In [10]:
excessiveIdling = {'totalElapsedTime':0,'count':0,'stops_count':0}

In [11]:
#calculated total idling times and its count
for keys,values in cleaned_idlingData.items():
    if ('ExcessiveIdlingElapsedTime' in keys):
        excessiveIdling['totalElapsedTime']+=int(values)
        excessiveIdling['count'] +=1

In [12]:
excessiveIdling

{'totalElapsedTime': 700, 'count': 5, 'stops_count': 0}

In [13]:
# function to convert timestamps to epoch format values
def convert_to_epoch_time(timestamps_dict):
    '''This function converts timestamps of given dictionary into epoch time and adds its elapsedIdlingTime to it.'''
    epoch_timestamps = {}
    ElapsedTime = 0
    for key, value in timestamps_dict.items():
        if 'ExcessiveIdlingElapsedTime' in key:
            ElapsedTime = int(value)
        elif 'TimeStamp' in key:
            timestamp = datetime.strptime(value, "%m/%d/%Y %I:%M:%S %p")
            epoch_timestamps[key] = int(timestamp.timestamp()) + ElapsedTime
    return epoch_timestamps

In [14]:
#converted into epoch format
epoch_data = convert_to_epoch_time(cleaned_idlingData)
print(epoch_data)

{'ExcessiveIdlingTimeStamp1': 1689333167, 'ExcessiveIdlingTimeStamp2': 1689333507, 'ExcessiveIdlingTimeStamp3': 1689334127, 'ExcessiveIdlingTimeStamp4': 1689335267, 'ExcessiveIdlingTimeStamp5': 1689335567}


In [15]:
#compare the epoch timestamps and if there is diff of less then a minute in two cont timestamps don't count is as a stop
count = 0
previousTimeStamp = None
for k,v in epoch_data.items():
    if (previousTimeStamp is not None) and (v-previousTimeStamp <61):
        previousTimeStamp = v
        print("Continuous minute timestamp, Skipping....")
        continue
    else:
        count+=1
        previousTimeStamp = v

In [16]:
excessiveIdling['stops_count'] = count

In [17]:
excessiveIdling

{'totalElapsedTime': 700, 'count': 5, 'stops_count': 5}

In [22]:
with open("idling_details.txt", 'w') as output:
    output.write(str(excessiveIdling))